In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# Cylinder combustion chamber
L = 75.77 / 1000
ID = 38.13 / 1000
OD = 100 / 1000

# Material properties matching Ansys - all of these change with temperature
rho = 8920
k = 391.1
Cp = 385

# Propellant and initial temperature
Ti = 298
Tfl = 2067

A = math.pi * ID * L # Internal chamber surface area [m2]
A2 = math.pi * OD * L # Outer chamber surface area [m2]

# Split the combustion chamber into equal segments
r = 20
sl_size = (OD - ID) / (2 * r)

# Calculate slice radii
sliceR = []
sliceD = []
V = []
m = []
for i in range(r):
    sliceR.append(ID / 2 + sl_size * (i + 1))
    d = ((ID / 2 + sl_size * (i + 1)) * 2)
    sliceD.append(d)
    if i == 0:
        v = math.pi * (d/2)**2 * L - math.pi * (ID/2)**2 * L
    else:
            v = math.pi * (d/2)**2 * L - math.pi * (sliceD[i - 1] / 2)**2 * L
    V.append(v) # Volume [m3]
    m.append(v * rho)

h_bartz = 4116.3 # Bartz
h_nat = 10

t = np.linspace(0, 5, num = 15000)

Ti = np.linspace(290, 290, num = r)

temps = pd.DataFrame({"t = 0": Ti})



for i in range(len(t)):
    if i == 0:
        continue
        
    else:
        # Start an empty list to record the temperature of each position for a specific moment in time
        T = []
        
        # Convection into first slice
        Q_conv = h_bartz * A * (Tfl - temps.iloc[0][i-1])
        dT_conv = Q_conv * (t[i] - t[i - 1]) / (m[0] * Cp)
        T_no_cond = temps.iloc[0][i-1] + dT_conv # Inner wall temperature if no heat is removed by conduction
        
        # Conduction from first slice into second
        dT_cond = T_no_cond - temps.iloc[1][i-1] # Temp difference between each slice
        Q_cond = (2 * math.pi * k * L * dT_cond) / (math.log(sliceD[1] / sliceD[0]))
        dT_slice1 = (Q_conv - Q_cond) * (t[i] - t[i - 1]) / (m[0] * Cp)
        T_slice1 = temps.iloc[0][i-1] + dT_slice1
        T.append(T_slice1)
        
        Q_cond_vec = []
        
        # Use another for loop to iterate through the centre slices where heat transfer modes are conduction only
        for j in range(r - 2):
            if j == 0:
                # Temperature change in slice x from conduction in only - no conduction out
                dT_slice_int = Q_cond * (t[i] - t[i - 1]) / (m[j + 1] * Cp) 
                T_cond_int = temps.iloc[j + 1][i - 1] + dT_slice_int 

                # Conduction out
                dT_cond2 = T_cond_int - temps.iloc[j + 2][i - 1]
                Q_cond2 = (2 * math.pi * k * L * dT_cond2) / (math.log(sliceD[j + 2] / sliceD[j + 1]))
                dT_slice2 = (Q_cond - Q_cond2) * (t[i] - t[i - 1]) / (m[j + 1] * Cp)
                T_slice2 = temps.iloc[j + 1][i - 1] + dT_slice2
                T.append(T_slice2)
                Q_cond_vec.append(Q_cond2)
                
            else:
                dT_slice_int = Q_cond_vec[j - 1] * (t[i] - t[i - 1]) / (m[j + 1] * Cp) 
                T_cond_int = temps.iloc[j + 1][i - 1] + dT_slice_int 

                # Conduction out
                dT_cond2 = T_cond_int - temps.iloc[j + 2][i - 1]
                Q_cond2 = (2 * math.pi * k * L * dT_cond2) / (math.log(sliceD[j + 2] / sliceD[j + 1]))
                Q_cond_vec.append(Q_cond2)
                dT_slice2 = (Q_cond_vec[j - 1] - Q_cond2) * (t[i] - t[i - 1]) / (m[j + 1] * Cp)
                T_slice2 = temps.iloc[j + 1][i - 1] + dT_slice2
                T.append(T_slice2)
            

        # Temp change in slice 3 from conduction - not accounting for natural convection leaving
        dT_slice3_int = Q_cond_vec[r - 3] * (t[i] - t[i - 1]) / (m[r - 1] * Cp)
        T_cond_int2 = temps.iloc[r - 1][i-1] + dT_slice3_int
        
        # Natural convection from slice 3 to freestream air
        Q_conv_nat = h_nat * A2 * (T_cond_int2 - 290)
        dT_slice3 = (Q_cond_vec[r - 3] - Q_conv_nat) * (t[i] - t[i - 1]) / (m[r - 1] * Cp)
        T_slice3 = temps.iloc[r - 1][i-1] + dT_slice3
        T.append(T_slice3)
        
        temps[f"t = {t[i]}"] = T
        
plt.figure()
for i in range(r):
    plt.plot(t, temps.iloc[i], label = f"Slice Temperature {i + 1}")
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.grid(visible=True, which='both', axis='both', color='lightsteelblue', linestyle='--', linewidth=0.5)


plt.figure()
plt.plot(t,temps.iloc[0])
plt.xlabel("Time (s)")
plt.ylabel("Inner Wall Temperature (K)")
plt.grid(visible=True, which='both', axis='both', color='lightsteelblue', linestyle='--', linewidth=0.5)

plt.figure()
plt.plot(t,temps.iloc[r-1)
plt.xlabel("Time (s)")
plt.ylabel("Outer Wall Temperature (K)")
plt.grid(visible=True, which='both', axis='both', color='lightsteelblue', linestyle='--', linewidth=0.5)